In [1]:
#CELL 1
import pandas as pd

# Read the Excel file (updated path since we're now in Joachim-Afrikaans folder)
df = pd.read_excel('../lexicon_6000 words.xlsx')

# Display basic information about the dataset
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
print(df.head())


Dataset shape: (6963, 8)
Columns: ['CILUBA', 'French', 'English', 'Afrikaans', 'Zulu', 'Score', 'Sentiment', 'Nature']

First few rows:
      CILUBA     French      English    Afrikaans              Zulu  Score  \
0      Akaja    Arrange      arrange     Rangskik             Hlela    1.0   
1  Akajilula  Rearrange    rearrange  Herrangskik     Hlela kabusha    1.0   
2      Akula      Parle        speak        Praat        Uyakhuluma    2.0   
3    Akulula    Reparle  speak again        Praat  Iphinde ikhulume    2.0   
4      Aluja      Remet   hands over        Hande        Isaphulelo    3.0   

  Sentiment Nature  
0   Positif  Verbe  
1   Positif  Verbe  
2   Positif  Verbe  
3   Positif  Verbe  
4   Positif  Verbe  


In [2]:
#CELL 2 - Google Cloud Translate API
import os

# Try to import Google Cloud Translate with helpful error message
try:
    from google.cloud import translate_v2 as translate
    print("✅ Google Cloud Translate imported successfully!")
    print(f"   Module location: {translate.__file__}")

    # Test that we can create a client (without authentication)
    try:
        # Just test the class exists, don't actually create it yet
        client_class = translate.Client
        print("✅ Translation client class available")
    except Exception as e:
        print(f"⚠️  Warning: Translation client issue: {e}")

except ImportError as e:
    print("❌ Google Cloud Translate not found!")
    print("To fix this issue, try ONE of these solutions:")
    print("")
    print("SOLUTION 1 - Install system-wide (recommended):")
    print("  python3 -m pip install google-cloud-translate --break-system-packages")
    print("")
    print("SOLUTION 2 - Use virtual environment:")
    print("  source venv_gcloud/bin/activate")
    print("  pip install google-cloud-translate")
    print("  # Then restart your Jupyter notebook from within the activated environment")
    print("")
    print("SOLUTION 3 - Install using conda (if you have it):")
    print("  conda install -c conda-forge google-cloud-translate")
    print("")
    print("After installation, restart your Jupyter kernel and try again.")
    raise e

# Set up Google Cloud credentials
service_account_key = 'skillful-hope-475413-d0-e1cdda43a60f.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key
print(f"🔐 Authentication configured with: {service_account_key}")

# Project ID
project_id = "inf791-translate-476018"

def translate_text(text, source_language='en', target_language='afr'):
    """
    Translate text using Google Cloud Translate API

    Args:
        text (str or list): Text to translate
        source_language (str): Source language code (default: 'en' for English)
        target_language (str): Target language code (default: 'ts' for Afrikaans)

    Returns:
        dict: Translation result
    """
    # Initialize the Translation client
    translate_client = translate.Client()

    # Prepare the request content
    if isinstance(text, str):
        contents = [text]
    elif isinstance(text, list):
        contents = text
    else:
        contents = [str(text)]

    try:
        # Perform the translation
        result = translate_client.translate(
            contents,
            source_language=source_language,
            target_language=target_language
        )

        return result

    except Exception as e:
        print(f"Translation error: {e}")
        return None

# Example usage with your specified content
print("=== Google Cloud Translate API Test ===")

# Your specified translation request
test_content = ["Hello"]
source_lang = "en"
target_lang = "af"

print(f"Translating: {test_content}")
print(f"From: {source_lang} (English)")
print(f"To: {target_lang} (Afrikaans)")
print(f"Project ID: {project_id}")

# Note: To run this cell successfully, you need to:
# 1. Set up authentication by placing your service account JSON file in the project
# 2. Set the GOOGLE_APPLICATION_CREDENTIALS environment variable
# 3. Uncomment and run the translation function below

# Test the translation with your service account
print("\n🚀 Testing translation...")
translation_result = translate_text(test_content, source_lang, target_lang)
if translation_result:
    for i, translation in enumerate(translation_result):
        print(f"Original: {test_content[i]}")
        print(f"Translated: {translation['translatedText']}")
        print(f"Detected source language: {translation.get('detectedSourceLanguage', 'N/A')}")
else:
    print("❌ Translation failed. Check your authentication and API setup.")

print("\n✅ Translation test completed!")


✅ Google Cloud Translate imported successfully!
   Module location: C:\Users\Mentz Family PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\google\cloud\translate_v2\__init__.py
✅ Translation client class available
🔐 Authentication configured with: skillful-hope-475413-d0-e1cdda43a60f.json
=== Google Cloud Translate API Test ===
Translating: ['Hello']
From: en (English)
To: af (Afrikaans)
Project ID: inf791-translate-476018

🚀 Testing translation...
Original: Hello
Translated: Hallo
Detected source language: N/A

✅ Translation test completed!


In [3]:
#CELL 3 - Authentication Setup (Run this first)
import os
import json

def setup_gcloud_auth(service_account_path=None):
    """
    Setup Google Cloud authentication

    Args:
        service_account_path (str): Path to your service account JSON file
    """

    print("=== Google Cloud Authentication Setup ===")

    if service_account_path and os.path.exists(service_account_path):
        # Set the environment variable for authentication
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_path
        print(f"✅ Authentication set up with service account: {service_account_path}")

        # Verify the service account file
        try:
            with open(service_account_path, 'r') as f:
                service_account = json.load(f)
                print(f"   Project ID: {service_account.get('project_id', 'Not found')}")
                print(f"   Client email: {service_account.get('client_email', 'Not found')}")
        except Exception as e:
            print(f"⚠️  Warning: Could not read service account file: {e}")

    else:
        print("📋 Authentication Setup Instructions:")
        print("1. Download your service account JSON file from Google Cloud Console")
        print("2. Place it in your project directory (keep it private!)")
        print("3. Update the path below and uncomment the setup line:")
        print("   # setup_gcloud_auth('path/to/your-service-account-key.json')")
        print("")
        print("🔐 Alternative: Set environment variable manually:")
        print("   os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'your-key-file.json'")
        print("")
        print("⚠️  IMPORTANT: Never commit your service account key to version control!")
        print("   Add your key file to .gitignore")

# Set up authentication with your service account key
service_account_key = 'skillful-hope-475413-d0-e1cdda43a60f.json'
setup_gcloud_auth(service_account_key)


=== Google Cloud Authentication Setup ===
✅ Authentication set up with service account: skillful-hope-475413-d0-e1cdda43a60f.json
   Project ID: skillful-hope-475413-d0
   Client email: id-91-assignment@skillful-hope-475413-d0.iam.gserviceaccount.com


In [4]:
#CELL 4 - Test with Exact JSON Format
import json

# Your specified JSON request format
request_json = {
    "contents": [
        "Hello"
    ],
    "sourceLanguageCode": "en",
    "targetLanguageCode": "af"
}

print("=== Testing with Your Specified JSON Format ===")
print("Request JSON:")
print(json.dumps(request_json, indent=2))
print()

# Test if the service account key file exists
service_account_key = 'skillful-hope-475413-d0-e1cdda43a60f.json'
if not os.path.exists(service_account_key):
    print(f"❌ Service account key not found: {service_account_key}")
    print("Please place your service account key file in this directory:")
    print(f"   {os.getcwd()}/")
    print("Then run this cell again.")
else:
    print(f"✅ Service account key found: {service_account_key}")

    # Set up authentication
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key

    try:
        from google.cloud import translate_v2 as translate

        # Create translation client
        translate_client = translate.Client()

        # Extract parameters from your JSON format
        contents = request_json["contents"]
        source_lang = request_json["sourceLanguageCode"]
        target_lang = request_json["targetLanguageCode"]

        print(f"📝 Translating: {contents}")
        print(f"🌍 From: {source_lang} → To: {target_lang}")
        print()

        # Perform translation using your exact parameters
        result = translate_client.translate(
            contents,
            source_language=source_lang,
            target_language=target_lang
        )

        print("🎉 Translation Results:")
        for i, translation in enumerate(result):
            print(f"  Original: '{contents[i]}'")
            print(f"  Translated: '{translation['translatedText']}'")
            if 'detectedSourceLanguage' in translation:
                print(f"  Detected Language: {translation['detectedSourceLanguage']}")
            print()

        # Show the full response format
        print("📋 Full API Response:")
        for i, translation in enumerate(result):
            response_json = {
                "translatedText": translation['translatedText'],
                "detectedSourceLanguage": translation.get('detectedSourceLanguage', source_lang)
            }
            print(json.dumps(response_json, indent=2))

    except Exception as e:
        print(f"❌ Translation error: {e}")
        print("This might be due to:")
        print("  - Authentication issues")
        print("  - API not enabled")
        print("  - Network connectivity")
        print("  - Invalid language code")

print("\n" + "="*50)


=== Testing with Your Specified JSON Format ===
Request JSON:
{
  "contents": [
    "Hello"
  ],
  "sourceLanguageCode": "en",
  "targetLanguageCode": "af"
}

✅ Service account key found: skillful-hope-475413-d0-e1cdda43a60f.json
📝 Translating: ['Hello']
🌍 From: en → To: af

🎉 Translation Results:
  Original: 'Hello'
  Translated: 'Hallo'

📋 Full API Response:
{
  "translatedText": "Hallo",
  "detectedSourceLanguage": "en"
}



In [5]:
#CELL 5 - Translate French to English and Save CSV
import pandas as pd
import os
import json
from google.cloud import translate_v2 as translate
import time

print("=== French to English Translation & CSV Export ===")
print("📊 Processing lexicon data for French → English translation")
print()

# Ensure service account key exists
service_account_key = 'skillful-hope-475413-d0-e1cdda43a60f.json'
if not os.path.exists(service_account_key):
    print(f"❌ Service account key not found: {service_account_key}")
    print("Please place your service account key file in this directory first.")
else:
    print(f"✅ Service account key found: {service_account_key}")

    # Set up authentication
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key

    try:
        # Initialize translation client
        translate_client = translate.Client()
        print("✅ Google Cloud Translate client initialized")

        # Read the Excel file (updated path since we're now in joachim-Afrikaans folder)
        excel_path = '../lexicon_6000 words.xlsx'
        df = pd.read_excel(excel_path)
        print(f"📋 Loaded dataset with {len(df)} rows from {excel_path}")

        # Check if 'French' column exists
        if 'French' not in df.columns:
            print(f"❌ 'French' column not found. Available columns: {list(df.columns)}")
        else:
            print(f"✅ Found 'French' column with {len(df)} entries")

            # Create a copy of the dataframe to work with
            df_translated = df.copy()

            # Get French words for translation
            french_words = df['French'].dropna().tolist()
            print(f"🔤 Found {len(french_words)} French words to translate")

            print("\n🚀 Starting translation process...")
            print("⏳ This may take a few minutes depending on the number of words...")

            # Translate in batches to avoid rate limits
            batch_size = 50  # Translate 50 words at a time
            total_words = len(french_words)
            all_translations = []

            for i in range(0, total_words, batch_size):
                batch_words = french_words[i:i + batch_size]
                batch_end = min(i + batch_size, total_words)

                print(f"📝 Translating batch {i//batch_size + 1}: words {i+1}-{batch_end} of {total_words}")

                try:
                    # Translate batch of words
                    results = translate_client.translate(
                        batch_words,
                        source_language='fr',  # French
                        target_language='en'   # English
                    )

                    # Store results
                    batch_translations = [result['translatedText'] for result in results]
                    all_translations.extend(batch_translations)

                    print(f"   ✅ Batch {i//batch_size + 1} completed successfully")

                    # Add small delay to respect rate limits
                    time.sleep(0.5)

                except Exception as e:
                    print(f"   ⚠️ Error in batch {i//batch_size + 1}: {e}")
                    # Add empty translations for failed batch
                    all_translations.extend([None] * len(batch_words))
                    continue

            print(f"\n✅ Translation completed! Processed {len(all_translations)} translations")

            # Apply translations to the dataframe
            print("📊 Applying translations to dataset...")
            # Ensure we have the right number of translations
            if len(all_translations) == len(df_translated[df_translated['French'].notna()]):
                df_translated.loc[df_translated['French'].notna(), 'English_Translation'] = all_translations
            else:
                print(f"⚠️ Translation count mismatch. Expected {len(df_translated[df_translated['French'].notna()])}, got {len(all_translations)}")
                df_translated['English_Translation'] = None

            # Count successful translations
            successful_translations = df_translated['English_Translation'].notna().sum()
            print(f"✅ Successfully translated {successful_translations} out of {len(df_translated)} entries")

            # Generate filename with timestamp for French to English
            from datetime import datetime
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f"lexicon_french_to_english_{timestamp}.csv"
            output_path = output_filename  # Save in current directory (joachim-Afrikaans)

            # Save to CSV
            df_translated.to_csv(output_path, index=False, encoding='utf-8')
            print(f"💾 CSV file saved successfully!")
            print(f"📂 Location: {output_path}")
            print(f"📊 File contains {len(df_translated)} rows and {len(df_translated.columns)} columns")

            # Show sample of translations
            print("\n📋 Sample Translations:")
            print("-" * 50)
            sample_df = df_translated[df_translated['English_Translation'].notna()].head(10)
            for idx, row in sample_df.iterrows():
                french_word = row['French']
                english_word = row['English_Translation']
                print(f"  {french_word:15} → {english_word}")

            if len(sample_df) < len(df_translated):
                print(f"  ... and {len(df_translated) - len(sample_df)} more translations")

            print("\n" + "="*60)
            print("🎉 TRANSLATION COMPLETE!")
            print(f"✅ Translated French words to English")
            print(f"💾 Saved CSV file: {output_path}")
            print(f"📊 Dataset: {len(df_translated)} rows, {len(df_translated.columns)} columns")
            print("="*60)

    except Exception as e:
        print(f"❌ Translation process failed: {e}")
        print("This might be due to:")
        print("  - Authentication issues")
        print("  - Google Translate API not enabled")
        print("  - Network connectivity problems")
        print("  - Rate limiting")


=== French to English Translation & CSV Export ===
📊 Processing lexicon data for French → English translation

✅ Service account key found: skillful-hope-475413-d0-e1cdda43a60f.json
✅ Google Cloud Translate client initialized
📋 Loaded dataset with 6963 rows from ../lexicon_6000 words.xlsx
✅ Found 'French' column with 6963 entries
🔤 Found 6954 French words to translate

🚀 Starting translation process...
⏳ This may take a few minutes depending on the number of words...
📝 Translating batch 1: words 1-50 of 6954
   ✅ Batch 1 completed successfully
📝 Translating batch 2: words 51-100 of 6954
   ✅ Batch 2 completed successfully
📝 Translating batch 3: words 101-150 of 6954
   ✅ Batch 3 completed successfully
📝 Translating batch 4: words 151-200 of 6954
   ✅ Batch 4 completed successfully
📝 Translating batch 5: words 201-250 of 6954
   ✅ Batch 5 completed successfully
📝 Translating batch 6: words 251-300 of 6954
   ✅ Batch 6 completed successfully
📝 Translating batch 7: words 301-350 of 6954
 

In [6]:
#CELL 6 - Translate English to Afrikaans and Save CSV
import pandas as pd
import os
import json
from google.cloud import translate_v2 as translate
import time

print("=== English to Afrikaans Translation & CSV Export ===")
print("📊 Processing lexicon data for English → Afrikaans translation")
print()

# Ensure service account key exists
service_account_key = 'skillful-hope-475413-d0-e1cdda43a60f.json'
if not os.path.exists(service_account_key):
    print(f"❌ Service account key not found: {service_account_key}")
    print("Please place your service account key file in this directory first.")
else:
    print(f"✅ Service account key found: {service_account_key}")

    # Set up authentication
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key

    try:
        # Initialize translation client
        translate_client = translate.Client()
        print("✅ Google Cloud Translate client initialized")

        # Find the most recent French-to-English CSV file
        import glob
        csv_pattern = 'lexicon_french_to_english_*.csv'
        csv_files = glob.glob(csv_pattern)

        if not csv_files:
            print(f"❌ No French-to-English CSV files found matching pattern: {csv_pattern}")
            print("Please run Cell 6 first to generate the French-to-English translations.")
            raise FileNotFoundError("Source CSV file not found")

        # Use the most recent file
        latest_csv = max(csv_files, key=os.path.getctime)
        print(f"📋 Using source file: {latest_csv}")

        # Read the CSV file with French-to-English translations
        df = pd.read_csv(latest_csv)
        print(f"📋 Loaded dataset with {len(df)} rows from {latest_csv}")

        # Check if 'English_Translation' column exists (from previous French-to-English step)
        if 'English_Translation' not in df.columns:
            print(f"❌ 'English_Translation' column not found. Available columns: {list(df.columns)}")
            print("This suggests the source file doesn't contain French-to-English translations.")
            print("Please run Cell 6 first to generate the French-to-English translations.")
        else:
            print(f"✅ Found 'English_Translation' column with {len(df)} entries")
            print("🔗 Using English translations from French as source for Afrikaans translation")

            # Create a copy of the dataframe to work with
            df_translated = df.copy()

            # Get English words from the translation column
            english_words = df['English_Translation'].dropna().tolist()
            print(f"🔤 Found {len(english_words)} English words to translate")
            print(f"📊 These are derived from French-to-English translations")

            print("\n🚀 Starting translation process...")
            print("⏳ This may take a few minutes depending on the number of words...")

            # Translate in batches to avoid rate limits
            batch_size = 50  # Translate 50 words at a time
            total_words = len(english_words)
            all_translations = []

            for i in range(0, total_words, batch_size):
                batch_words = english_words[i:i + batch_size]
                batch_end = min(i + batch_size, total_words)

                print(f"📝 Translating batch {i//batch_size + 1}: words {i+1}-{batch_end} of {total_words}")

                try:
                    # Translate batch of words
                    results = translate_client.translate(
                        batch_words,
                        source_language='en',  # English
                        target_language='af'   # Afrikaans
                    )

                    # Store results
                    batch_translations = [result['translatedText'] for result in results]
                    all_translations.extend(batch_translations)

                    print(f"   ✅ Batch {i//batch_size + 1} completed successfully")

                    # Add small delay to respect rate limits
                    time.sleep(0.5)

                except Exception as e:
                    print(f"   ⚠️ Error in batch {i//batch_size + 1}: {e}")
                    # Add empty translations for failed batch
                    all_translations.extend([None] * len(batch_words))
                    continue

            print(f"\n✅ Translation completed! Processed {len(all_translations)} translations")

            # Apply translations to the dataframe
            print("📊 Applying translations to dataset...")
            # Ensure we have the right number of translations
            if len(all_translations) == len(df_translated[df_translated['English_Translation'].notna()]):
                df_translated.loc[df_translated['English_Translation'].notna(), 'Afrikaans_Translation'] = all_translations
            else:
                print(f"⚠️ Translation count mismatch. Expected {len(df_translated[df_translated['English_Translation'].notna()])}, got {len(all_translations)}")
                df_translated['Afrikaans_Translation'] = None

            # Count successful translations
            successful_translations = df_translated['Afrikaans_Translation'].notna().sum()
            print(f"✅ Successfully translated {successful_translations} out of {len(df_translated)} entries")

            # Generate filename with timestamp for English to Afrikaans
            from datetime import datetime
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f"lexicon_english_to_Afrikaans_{timestamp}.csv"
            output_path = output_filename  # Save in current directory (joachim-Afrikaans)

            # Save to CSV
            df_translated.to_csv(output_path, index=False, encoding='utf-8')
            print(f"💾 CSV file saved successfully!")
            print(f"📂 Location: {output_path}")
            print(f"📊 File contains {len(df_translated)} rows and {len(df_translated.columns)} columns")

            # Show sample of translations (French → English → Afrikaans chain)
            print("\n📋 Sample Translation Chain (French → English → Afrikaans):")
            print("-" * 70)
            sample_df = df_translated[df_translated['Afrikaans_Translation'].notna()].head(10)
            for idx, row in sample_df.iterrows():
                french_word = row['French']
                english_word = row['English_Translation']
                Afrikaans_word = row['Afrikaans_Translation']
                print(f"  {french_word:12} → {english_word:15} → {Afrikaans_word}")

            if len(sample_df) < len(df_translated):
                print(f"  ... and {len(df_translated) - len(sample_df)} more translations")

            print("\n" + "="*60)
            print("🎉 TRANSLATION CHAIN COMPLETE!")
            print(f"✅ French → English → Afrikaans translation chain completed")
            print(f"💾 Saved CSV file: {output_path}")
            print(f"📊 Dataset: {len(df_translated)} rows, {len(df_translated.columns)} columns")
            print(f"🔗 Final CSV contains: French, English, English_Translation, Afrikaans_Translation")
            print("="*60)

    except Exception as e:
        print(f"❌ Translation process failed: {e}")
        print("This might be due to:")
        print("  - Authentication issues")
        print("  - Google Translate API not enabled")
        print("  - Network connectivity problems")
        print("  - Rate limiting")
        print("  - Afrikaans language support limitations")


=== English to Afrikaans Translation & CSV Export ===
📊 Processing lexicon data for English → Afrikaans translation

✅ Service account key found: skillful-hope-475413-d0-e1cdda43a60f.json
✅ Google Cloud Translate client initialized
📋 Using source file: lexicon_french_to_english_20251103_211636.csv
📋 Loaded dataset with 6963 rows from lexicon_french_to_english_20251103_211636.csv
✅ Found 'English_Translation' column with 6963 entries
🔗 Using English translations from French as source for Afrikaans translation
🔤 Found 6854 English words to translate
📊 These are derived from French-to-English translations

🚀 Starting translation process...
⏳ This may take a few minutes depending on the number of words...
📝 Translating batch 1: words 1-50 of 6854
   ✅ Batch 1 completed successfully
📝 Translating batch 2: words 51-100 of 6854
   ✅ Batch 2 completed successfully
📝 Translating batch 3: words 101-150 of 6854
   ✅ Batch 3 completed successfully
📝 Translating batch 4: words 151-200 of 6854
   ✅ 